# Utilities for Optimization

> This module handles all aspects of the world model, including state representation, environment dynamics, and prediction.

In [ ]:
#| default_exp optimizers.utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore import *
from fastcore.utils import *

In [ ]:
#| export
from functools import partial
from torch.optim import Optimizer

In [ ]:
#| export
import importlib
def get_cls(module_name, class_name):
    module = importlib.import_module(module_name)
    return getattr(module, class_name)

In [ ]:
#| export
def init_opt(cfg, model):
    optimizer_cls = get_cls("torch.optim", cfg.optimizer.name)
    optimizer = optimizer_cls(model.parameters(), lr=cfg.optimizer.lr)
    return optimizer

In [ ]:
#| export
# Source - https://stackoverflow.com/a
# Posted by isle_of_gods, modified by community. See post 'Timeline' for change history
# Retrieved 2025-11-15, License - CC BY-SA 4.0

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


In [ ]:
#| export
# from mawm.optimizers.schedulers import WSDSchedule, CosineWDSchedule
import torch
def init_opt(
    cfg,
    *models,
    betas=(0.9, 0.999),
    eps=1e-8,
):
    all_params = []
    for model in models:
        all_params += list(model.parameters())
    

    optimizer = torch.optim.AdamW(all_params, betas=betas, eps=eps)
    # scheduler = WSDSchedule(
    #     optimizer,
    #     warmup_steps=int(warmup * iterations_per_epoch),
    #     anneal_steps=int(anneal * iterations_per_epoch),
    #     start_lr=start_lr,
    #     ref_lr=ref_lr,
    #     final_lr=final_lr,
    #     T_max=int(num_epochs * iterations_per_epoch),
    # )
    # wd_scheduler = CosineWDSchedule(
    #     optimizer,
    #     ref_wd=wd,
    #     final_wd=final_wd,
    #     T_max=int(num_epochs * iterations_per_epoch),
    # )
    # scaler = torch.cuda.amp.GradScaler() if mixed_precision else None
    return optimizer#, scaler, scheduler, wd_scheduler


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()